In [32]:
import pandas as pd
import numpy as np
import scipy as sp

#ret = pd.read_csv('omni.csv') FUCK relative paths
ret = pd.read_excel('/Users/zoakes/Desktop/Quant/Py_Review/Omni/Omni.xlsx') #THANK FUCKING GOD -- just drag drop into terminal !


In [8]:
#Monte Carlo -- Portfolio Optimization
#np.random.seed(100)

##INCLUDE DATA HERE 
'''Test Case
#CONCAT RETURNS TOGETHER below
#pd.concat([system,system2,system3],axis=1)
#sys.columns = ['VX','RB','NQ']


#sys.pct_change(1).head()
'''
#Normalize returns--Nat Log
log_ret = np.log(ret/ret.shift(1))
log_ret.head()

#Create Temporary (Random) weights
weights = np.array(np.random.random(26))

#Rebalance w/ constraints
weights = weights/np.sum(weights)
print(weights)

num_runs = 1000    #Kick this number up 10k-100k for better results
all_weights = np.zeros((num_runs,len(ret.columns)))
ret_arr = np.zeros(num_runs)
vol_arr = np.zeros(num_runs)
sharpe_arr = np.zeros(num_runs)
corr_arr = np.zeros(num_runs)
b_arr = np.zeros(num_runs)

for run in range(num_runs):
    
    #Weights
    weights = np.array(np.random.random(26))  #CHG to number securities
    weights = weights/np.sum(weights)
    
    #Save weights
    all_weights[run,:] = weights
    
    #Expected Ret
    exp_ret = np.sum((log_ret.mean() * weights) * 252)
    ret_arr[run] = np.sum( (log_ret.mean() * weights) * 252) #why T -> year?
    
    #Exp Vol: (Lets attempt some linear algebra w/out runtime error!)
    exp_vol = np.sum((log_ret.std() * weights) * 252)
    #       Sqrt of dot product of Transposed weights X Cov of Log returns & weights--whew.
    vol_arr[run] = np.sqrt(np.dot(weights.T,np.dot(log_ret.cov()*252,weights)))
    
    #Sharpe
    SR = exp_ret/exp_vol
    sharpe_arr[run] = ret_arr[run]/vol_arr[run]
    
    #Correlation
    t_corr = log_ret.corr() #[1:] #Removes benchmark--if included
    t_corr2 = (t_corr.sum()-1)/t_corr.count()
    corr_arr[run] = np.sum(t_corr2 * weights)
    s_corr = np.sum((log_ret.corr() * weights)) # THIS HAD [1:] (for BM/Beta) bw final parentheses.#Use this one, although above is more accurate/complex
    
    #Beta 
    #Should really add a concat here, to include the benchmark, although painfully slow.
    #s_beta = np.sum((log_ret.cov()[1:] * weights)
    b_arr[run] = np.sum(np.sum(log_ret.cov() * weights)) # THIS HAD [1] bw final parentheres.#This looks right -- if not, remove the [1]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in log
  


[0.04585219 0.04814204 0.01057791 0.02100759 0.01548583 0.07033481
 0.05312662 0.0156017  0.05117976 0.03438495 0.04575009 0.01960094
 0.06543043 0.07221424 0.03375906 0.04763324 0.03454841 0.02143607
 0.01181227 0.04536916 0.07622889 0.02588129 0.04184965 0.00935568
 0.04705343 0.03638381]


In [10]:
#sharpe_arr.argmax()
#b_arr.argmin()
corr_arr.argmin()

462

In [11]:
all_weights[462,:]

array([0.04218303, 0.05204915, 0.01169955, 0.04986516, 0.0885978 ,
       0.00218859, 0.04290493, 0.08858309, 0.04813855, 0.08930225,
       0.01626552, 0.01436139, 0.02866229, 0.03736161, 0.01276935,
       0.06953922, 0.03072552, 0.00917211, 0.00615338, 0.09121499,
       0.05331602, 0.01324064, 0.01288595, 0.0042662 , 0.00561321,
       0.07894053])

In [46]:
def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T,np.dot(log_ret.cov()*252,weights)))
    sr = ret/vol
    corr = np.sum(np.sum(log_ret.corr() * weights)) #Had [1:] bw parenth's #Removes benchmark... should do better
    b = np.sum(np.sum(log_ret.cov() * weights)[1])   #Selects only benchmark cov(if included for Beta calc)... this could be done better.
    return np.array([ret,vol,sr,corr,b])

In [71]:
#SLS Minimize Optimization (for larger datasets--or 12 instrument portfolios!)
from scipy.optimize import minimize

def neg_sharpe(weights):
    return get_ret_vol_sr(weights)[2]*-1

def check_sum(weights):
    return np.sum(weights)-1

def get_c(weights):
    t_corr = log_ret.corr() #[1:] #Removes benchmark--if included
    t_corr2 = (t_corr.sum()-1)/t_corr.count()
    corr = np.sum(t_corr2 * weights)
    return corr
    #return get_ret_vol_sr(weights)[3]*-1    #This doesnt make sense, but it works.

def opt_beta(weights):
    return get_ret_vol_sr(weights)[4]*-1    #Again, this doesnt make sense, but it works.

cons = ({'type':'eq','fun':check_sum})

bounds = ((0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1))

#init_guess = [.1,.1,.,.20]
init_guess = np.array(np.random.random(26))  #CHG to number securities
init_guess = weights/np.sum(weights)

opt_results = minimize(get_c,init_guess,method='SLSQP',bounds=bounds,constraints=cons)

In [80]:
wts = opt_results.x

#res = [(i,j) for i in opt_results.x for j in ret.columns]
#res #Why did this run 100x? huh

res = {}
for value in wts:
    for key in ret.columns:
        res[key] = value
        

print('MC Sim: Painfully slow')
%timeit -n50 corr_arr.argmin()

%timeit -n50 opt_results,opt_results.x
print('SP.Opt.Min: Optimized to < 750ns')
print('Selection Confirmed -- Matches equities used (aside from AMZN)')
wts,ret.columns
#res

nums = {}
for value in wts:
    for key in ret.columns:
        nums[key] = value
wts,ret.columns

MC Sim: Painfully slow
The slowest run took 13.08 times longer than the fastest. This could mean that an intermediate result is being cached.
4.85 µs ± 6.94 µs per loop (mean ± std. dev. of 7 runs, 50 loops each)
830 ns ± 48.6 ns per loop (mean ± std. dev. of 7 runs, 50 loops each)
SP.Opt.Min: Optimized to < 750ns
Selection Confirmed -- Matches equities used (aside from AMZN)


(array([0.00000000e+00, 1.10176778e-16, 0.00000000e+00, 4.44089210e-16,
        1.00000000e+00, 0.00000000e+00, 3.89642841e-17, 0.00000000e+00,
        2.18144561e-15, 0.00000000e+00, 3.80961079e-17, 5.42103978e-17,
        6.88808549e-17, 0.00000000e+00, 3.08580942e-18, 5.08267895e-16,
        0.00000000e+00, 0.00000000e+00, 3.72275907e-17, 0.00000000e+00,
        8.46728473e-18, 0.00000000e+00, 0.00000000e+00, 1.42717591e-16,
        1.73552225e-17, 0.00000000e+00]),
 Index(['AAPL', 'ADBE', 'AMZN', 'APC', 'BA', 'BABA', 'BAC', 'CAT', 'COST',
        'CMG', 'CRM', 'CVS', 'GOOGL', 'GS', 'INTC', 'KMI', 'LULU', 'NFLX',
        'NVDA', 'PYPL', 'S', 'SLB', 'TEVA', 'TSLA', 'UAL', 'UNH'],
       dtype='object'))

In [255]:
fin_ret = ret[['ADBE','APC','CMG','COST','GOOGL','GS','KMI','NFLX','PYPL','SLB','TEVA','UAL']]
fin_ret.head()

,ADBE,APC,CMG,COST,GOOGL,GS,KMI,NFLX,PYPL,SLB,TEVA,UAL
0,0.0034,0.0001,0.0005,0.0040,0.0000,-0.0036,-0.0011,-0.0013,0.0010,0.0012,0.0032,0.0021
1,0.0025,0.0002,0.0011,0.0006,0.0017,0.0008,0.0000,0.0002,0.0002,-0.0014,0.0003,0.0020
2,0.0003,-0.0008,0.0017,-0.0004,0.0029,0.0012,0.0018,0.0009,-0.0014,0.0030,0.0031,0.0003
3,0.0011,-0.0004,0.0022,0.0011,0.0010,0.0022,0.0008,0.0025,0.0009,0.0036,0.0000,0.0011
4,0.0003,0.0003,0.0026,-0.0010,-0.0001,-0.0003,-0.0016,0.0000,0.0005,0.0011,0.0030,0.0038


In [263]:
#Scipy optimize min (on final values)
#Weights

def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = np.sum(fin_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T,np.dot(fin_ret.cov()*252,weights)))
    sr = ret/vol
    corr = np.sum(np.sum(fin_ret.corr() * weights)) #Had [1:] bw parenth's #Removes benchmark... should do better
    b = np.sum(np.sum(fin_ret.cov() * weights)[1])   #Selects only benchmark cov(if included for Beta calc)... this could be done better.
    return np.array([ret,vol,sr,corr,b])


weights = np.array(np.random.random(12))  #CHG to number securities
weights = weights/np.sum(weights)

init_guess = np.array(np.random.random(12))  #CHG to number securities
init_guess = weights/np.sum(weights)


def neg_sharpe(weights):
    return get_ret_vol_sr(weights)[2]*-1

def check_sum(weights):
    return np.sum(weights)-1

def get_c(weights):
    t_corr = fin_ret.corr() #[1:] #Removes benchmark--if included
    t_corr2 = (t_corr.sum()-1)/t_corr.count()
    corr = np.sum(t_corr2 * weights)
    return corr#*-1
    #return get_ret_vol_sr(weights)[3]*-1    #This doesnt make sense, but it works.

def opt_beta(weights):
    return get_ret_vol_sr(weights)[4]*-1    #Again, this doesnt make sense, but it works.

cons = ({'type':'eq','fun':check_sum})


bounds = ((0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1),(0,1))

opt_res = minimize(get_c,weights,method='SLSQP',bounds=bounds,constraints=cons)

In [103]:
fin_wts = opt_res.x
fin_wts,fin_ret.columns

(array([1.00000000e+00, 0.00000000e+00, 1.11022302e-16, 4.64580631e-16,
        6.93889390e-17, 0.00000000e+00, 4.51028104e-17, 0.00000000e+00,
        0.00000000e+00, 4.16333634e-17, 1.83186799e-15, 0.00000000e+00]),
 Index(['ADBE', 'APC', 'CMG', 'COST', 'GOOGL', 'GS', 'KMI', 'NFLX', 'PYPL',
        'SLB', 'TEVA', 'UAL'],
       dtype='object'))

In [264]:
opt_res.x

array([1.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.77555756e-17, 3.81639165e-17, 1.14491749e-16,
       0.00000000e+00, 2.77555756e-17, 0.00000000e+00, 1.68159757e-16])

In [265]:
#THIS BREAKS THE LAST MIN FUNCTION --- RE RUN THE PANDA FUNCTION ABOVE THE PRIOR MIN
fin_ret = fin_ret[['ADBE','APC','KMI','TEVA']]

wts = np.array(np.random.random(4))
wts = wts/np.sum(wts)

bounds = ((0,1),(0,1),(0,1),(0,1))

fin = minimize(get_c,wts,method='SLSQP',bounds=bnds,constraints=cons)
f = fin.x
f

array([6.9388939e-18, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00])

In [285]:
def get_nSmallest_corr(df,n):
    df = df.corr()   #input cor matrix, or pct chg
    #Unhide ^ if simply returns in corr, otherwise -- keep !
    sum_corr = (df.sum()-1)/df.count()
    return sum_corr.sort_values()[:n]
 

def get_nSmallest_beta(cov_mat,n):
    #cov_mat = cov_mat.cov()
    #input of Cov matrix,(get_beta()) OR ^ unhde, pct chg w/ BM in col 0
    cov_s = cov_mat['SPY'][1:]
    return cov_s.sort_values()[:5]




#covar = a.cov(b)
def get_beta(df,mkt,col_names=None):
    '''Ensure Benchmark column is named SPY, or include col names in list, BM first!'''
    rets = pd.concat([mkt,df],axis=1)
    if col_names != None: rets.columns = col_names #Change this to the df name 
    cv_mat = rets.cov()
    return cv_mat

print(get_nSmallest_corr(log_ret,26))
print('My correlations were simply off !')
cors = get_nSmallest_corr(log_ret,26)
pf_corr = cors[2]+cors[3]+cors[5]+cors[6]+cors[7]+cors[8]+cors[10]+cors[14]+cors[15]+cors[17]+cors[20]+cors[21]
pf_corr

BA      -0.089596
APC     -0.072699
COST    -0.064801
S       -0.051461
KMI     -0.049946
UAL     -0.045191
TEVA    -0.036906
GOOGL   -0.036856
CVS     -0.034662
PYPL    -0.026855
BABA    -0.024977
CAT     -0.022541
TSLA    -0.020793
GS      -0.018348
CMG     -0.016478
UNH     -0.013068
ADBE    -0.001329
NVDA     0.007544
CRM      0.018718
SLB      0.025028
NFLX     0.025336
LULU     0.026979
INTC     0.027074
BAC      0.028818
AAPL     0.036171
AMZN     0.072039
dtype: float64
My correlations were simply off !


-0.26454052598223193

In [68]:
#b = log_ret.cov().sum()[0]
#print(b)

'''R2 '''
import pandas_datareader as pdr

print(ret['COST'].shape[0])
#ret #38 rows 
cost = pdr.DataReader('cost',data_source='yahoo',start='2019-05-01',end='2019-06-23')
cost.shape[0]

#Costco security
cpct = cost['Adj Close'].pct_change()

#Should be omni
co = ret['COST'].astype(float)
#print(co,cpct)
data = [cpct,co]
#rs = pd.concat(data,axis=1)
'''Merge not working! why'''
#co['Sec'] = ret['COST']



#df = pd.DataFrame({'sec':cpct,'omni':co})
#df.head()
cc = cpct.fillna(0).astype(float).reset_index().drop(columns='Date')
co

data2 = [cc,co]
r2 = pd.concat(data2,axis=1)
RSquared = abs(r2.corr().iloc[0,1])
RSquared

def calcRSquared(ticker):
    if ticker not in ret.columns: return 'Not in Omni performance'
    sec = pdr.DataReader('TSLA',data_source='yahoo',start='2019-04-30',end='2019-06-21')
    om = ret[ticker]
    sec = sec['Adj Close'].pct_change().reset_index().drop(columns='Date')
    rdf = pd.concat([om,sec],axis=1)
    r2 = round(rdf.corr().iloc[0,1],4)
    return r2



ts = calcRSquared('TSLA')
gs = calcRSquared('GS')
ret.columns
gl = calcRSquared('GOOGL')
pl = calcRSquared('PYPL')
it = calcRSquared('INTC')
lu = calcRSquared('LULU')

38


In [79]:
print('GS {}, GOOGL {}, PYPL {} INTC {}, LULU {}'.format(gs,gl,pl,it,lu))
calcRSquared('ADBE') #FUCKING 20! JESUS

GS 0.0241, GOOGL -0.1455, PYPL -0.0986 INTC 0.0265, LULU 0.0198


0.2084